In [9]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
key = os.getenv("OPENAI_API_KEY")

In [11]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.3)